## Upload and unzip tensorflow model

In [1]:
!unzip text_classifier_model_tf.zip

Archive:  text_classifier_model_tf.zip
   creating: text_classifier_model/
   creating: text_classifier_model/1/
  inflating: text_classifier_model/1/saved_model.pb  
   creating: text_classifier_model/1/assets/
   creating: text_classifier_model/1/variables/
  inflating: text_classifier_model/1/variables/variables.index  
  inflating: text_classifier_model/1/variables/variables.data-00000-of-00001  


In [2]:
!wget https://github.com/futurexskill/ml-model-deployment/raw/main/tfidfmodel.pickle

--2021-04-20 15:12:31--  https://github.com/futurexskill/ml-model-deployment/raw/main/tfidfmodel.pickle
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/tfidfmodel.pickle [following]
--2021-04-20 15:12:31--  https://raw.githubusercontent.com/futurexskill/ml-model-deployment/main/tfidfmodel.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49660 (48K) [application/octet-stream]
Saving to: ‘tfidfmodel.pickle’

tfidfmodel.pickle   100%[===================>]  48.50K  --.-KB/s    in 0.01s   

2021-04-20 15:12:31 (4.07 MB/s) - ‘tfidfmodel.pickle’ saved [49660/

In [3]:
!ls

sample_data	       text_classifier_model_tf.zip
text_classifier_model  tfidfmodel.pickle


## Load tensorflow model

In [15]:
import tensorflow as tf
from tensorflow.keras.models import load_model

In [16]:
model = load_model('text_classifier_model/1/')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 500)               234000    
_________________________________________________________________
dense_1 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1002      
Total params: 485,502
Trainable params: 485,502
Non-trainable params: 0
_________________________________________________________________


## Load tfidf model

In [18]:
import pickle

In [19]:
with open('tfidfmodel.pickle', 'rb') as file:
  tfidf = pickle.load(file)

## Flask server

In [20]:
!pip install flask-ngrok

In [21]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request

In [22]:
app = Flask(__name__)
run_with_ngrok(app)

In [49]:
@app.route('/answer', methods=["POST"], endpoint='answer')
def text_classifier():
    request_data = request.get_json(force=True)
    text = request_data['sentence']
    print(f'Sentence: {text}')
    text_list = []
    text_list.append(text)
    print(text_list)
    numeric_text = tfidf.transform(text_list).toarray()
    output = model.predict(numeric_text)
    print(f"Prediction: {output}")
    sentiment = 'unknown'
    if output[0][0] > 0.5:
      print('Negative prediction')
      sentiment = 'negative'
    else:
      print('Positive prediction')
      sentiment = 'positive'
    print(f"Prediction: {sentiment}")
    return f"The prediction is {sentiment}"

In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d5494751be4d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Apr/2021 15:28:41] "POST /answer HTTP/1.1" 200 -


Sentence: Bad perfomance by India
['Bad perfomance by India']
Prediction: [[1.0000000e+00 3.6861845e-08]]
Negative prediction
Prediction: negative


127.0.0.1 - - [20/Apr/2021 15:28:59] "POST /answer HTTP/1.1" 200 -


Sentence: good perfomance by India
['good perfomance by India']
Prediction: [[0.2098132 0.7901868]]
Positive prediction
Prediction: positive
